In [ ]:
# Importare le librerie necessarie
import numpy as np
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
import os

import warnings
warnings.filterwarnings('ignore')


importance_list=[]
first_ord=[]
score_list = []
pvalue_list =[]
mse_list = []
mae_list = []
ypred=[]
ytest=[]

import json

with open('configAMPSIT.json') as config_file:
    config = json.load(config_file)

totalhours = config['totalhours']
variables = config['variables']
regions = config['regions']
verticalmax = config['verticalmax']
totalsim = config['totalsim']
parameter_names = config['parameter_names']
output_path = config['output_pathname']
tun_iter = config['tun_iter']
    
#############################################################################################         

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr

def sa_randomforest(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f):
  from sklearn.ensemble import RandomForestRegressor
  from joblib import dump, load
  
  if tun==1:

    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score

    rf = RandomForestRegressor()

    params = {
        'n_estimators': (10, 30),
        'max_depth': (2, 6),
        'min_samples_split': (2, 5),
        'min_samples_leaf': (1, 5),
        'max_features': ('sqrt', 'log2')
    }


    opt = BayesSearchCV(rf, params, n_iter=tun_iter, cv=5)
    opt.fit(X_train, y_train)
    
    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_rf_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_rf_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_rf_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))


    best_params = opt.get_params()
    best_rf_params = {key.replace("estimator__", ""): value for key, value in best_params.items() if key.startswith('estimator__')}
    rf = RandomForestRegressor(**best_rf_params)

    rf.fit(X_train, y_train)
    
    y_pred = rf.predict(X_test)

    importances = rf.feature_importances_
    
    sum_importances = np.sum(importances, axis=0)
    normalized_importances = importances / sum_importances

    dump(rf, output_path+'rf_model_'+f[:-4]+'.joblib')
    
  else:
    
    if tun==2:
      loaded_model = load(output_path+'rf_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      importances = loaded_model.feature_importances_
    
    
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances  
      
    else:            

      rf = RandomForestRegressor(n_estimators=20, max_depth=5, max_features='log2',min_samples_leaf= 1, min_samples_split= 2)
    
      rf.fit(X_train, y_train)
      
      y_pred = rf.predict(X_test)

      importances = rf.feature_importances_
      
      
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances

  if len(importance_list) <= totalhours-1:
      importance_list.append(importances)
      #importance_list.append(normalized_importances)
  else:
      importance_list = []               
  

  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
          
  from sklearn.metrics import mean_squared_error
  mse = mean_squared_error(y_test, y_pred)
  from sklearn.metrics import mean_absolute_error
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]


def sa_xgboost(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f):
  import xgboost as xgb
  from xgboost import XGBRFRegressor
  from joblib import dump, load
  
  xgb.set_config(verbosity=0)

  if tun==1:

    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score

    gb = XGBRFRegressor()

    params = {
        'n_estimators': (10, 30),
        'max_depth': (2, 6),
        'min_samples_split': (2, 5),
        'min_samples_leaf': (1, 5),
        'max_features': ('sqrt', 'log2')
    }

    opt = BayesSearchCV(gb, params, n_iter=tun_iter, cv=5)

    opt.fit(X_train, y_train)
    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_xgb_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_xgb_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_xgb_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))

    best_params = opt.get_params()
    gb = XGBRFRegressor(**best_params)

    gb.fit(X_train, y_train)
  
    y_pred = gb.predict(X_test)
    
    importances = gb.feature_importances_

    dump(gb, output_path+'xgb_model_'+f[:-4]+'.joblib')
    
  else:

    if tun==2:
      loaded_model = load(output_path+'xgb_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      importances = loaded_model.feature_importances_
      
    else:        
    
      gb = XGBRFRegressor(n_estimators=20, max_depth=5, max_features='log2',min_samples_leaf= 1, min_samples_split= 2)
    
      gb.fit(X_train, y_train)
    
      y_pred = gb.predict(X_test)

      importances = gb.feature_importances_

      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances

  if len(importance_list) <= totalhours-1:
      importance_list.append(importances)
  else:
      importance_list = []               

  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
            
  from sklearn.metrics import mean_squared_error
  mse = mean_squared_error(y_test, y_pred)
  from sklearn.metrics import mean_absolute_error
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]


def sa_cart(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f):
  from sklearn.tree import DecisionTreeRegressor
  from joblib import dump, load

  if tun==1:

    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score

    dt = DecisionTreeRegressor()

    params = {
        'max_depth': (2, 6),
        'min_samples_split': (2, 5),
        'min_samples_leaf': (1, 5)
    }

    opt = BayesSearchCV(dt, params, n_iter=tun_iter, cv=5)

    opt.fit(X_train, y_train)
    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_cart_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_cart_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_cart_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))

    best_params = opt.get_params()
    dt_params = {key: value for key, value in best_params.items() if key in ['max_depth', 'min_samples_split', 'min_samples_leaf']}

    dt = DecisionTreeRegressor(**dt_params)

    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)

    importances = dt.feature_importances_

    dump(dt, output_path+'cart_model_'+f[:-4]+'.joblib')
    
  else:
    
    if tun==2:
      loaded_model = load(output_path+'cart_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      importances = loaded_model.feature_importances_
      
    else:    
    
      dt = DecisionTreeRegressor(max_depth=5,min_samples_leaf= 1, min_samples_split= 2)
    
      dt.fit(X_train, y_train)

      y_pred = dt.predict(X_test)

      importances = dt.feature_importances_

  if len(importance_list) <= totalhours-1:
      importance_list.append(importances)
  else:
      importance_list = []               
  
  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
  
  from sklearn.metrics import mean_squared_error
  mse = mean_squared_error(y_test, y_pred)
  from sklearn.metrics import mean_absolute_error
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]


def sa_lassoregression(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f):
  import numpy as np
  from sklearn.linear_model import LassoCV
  from joblib import dump, load
 
  if tun==1:
    
    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score
    
    lasso_cv = LassoCV(cv=5)

    params = {
        'eps': (1e-5, 1e-1),
        'n_alphas': (50, 200),
        'tol': (1e-6, 1e-3)
    }

    opt = BayesSearchCV(lasso_cv, params, n_iter=tun_iter, cv=5)

    opt.fit(X_train, y_train)

    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_lasso_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_lasso_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_lasso_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))

    y_pred = opt.best_estimator_.predict(X_test)

    importances = np.abs(opt.best_estimator_.coef_)

    dump(opt, output_path+'lasso_model_'+f[:-4]+'.joblib')

  else:
  
    if tun==2:
      loaded_model = load(output_path+'lasso_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      importances = np.abs(loaded_model.coef_)
      
    else:
  
  
      lasso_cv = LassoCV(cv=5)
      
      lasso_cv.fit(X_train, y_train)                        

      y_pred = lasso_cv.predict(X_test)
      
      importances = np.abs(lasso_cv.coef_)           
                      
  if len(importance_list) <= totalhours-1:
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances       
      importance_list.append(normalized_importances)
  else:
      importance_list = []

  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
  
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]

def sa_svm(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f):
  from sklearn import svm
  
  from sklearn.model_selection import train_test_split, cross_val_score 
  from joblib import dump, load
  
  if tun==1:
    
    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score
    
    svm_model = svm.SVR()

    params = {
        'C': (1e-3, 1e+3, 'log-uniform'),
        'kernel': ['linear'],
        'gamma': (1e-3, 1e+1, 'log-uniform'),
        'epsilon': (1e-4, 1e-1, 'log-uniform'),
    }

    opt = BayesSearchCV(svm_model, params, n_iter=tun_iter, cv=5)

    opt.fit(X_train, y_train)

    score = cross_val_score(opt, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_svm_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_svm_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_svm_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(opt.best_params_))
        file.write("Cross-validation score: {}\n".format(score))

    y_pred = opt.predict(X_test)
    coef = opt.best_estimator_.coef_[0]  
    
    dump(opt, output_path+'svm_model_'+f[:-4]+'.joblib')
  
  else:
 
    if tun==2:
      loaded_model = load(output_path+'svm_model_'+f[:-4]+'.joblib')
      
      y_pred = loaded_model.predict(X_test)
      
      coef=loaded_model.coef_[0]
      
    else:
      
      clf = svm.SVR(kernel='linear', C=1, epsilon=0.1)
      clf.fit(X_train, y_train)
      
      y_pred = clf.predict(X_test)
      
      coef=clf.coef_[0]

  
  if len(importance_list) <= totalhours-1:
      sum_importances = np.sum(np.abs(coef))
      normalized_importances = coef / sum_importances      
      importance_list.append(normalized_importances)
  else:
      importance_list = []

  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]


def sa_baesyanreg(X_train, X_test, y_train, y_test,problem,N, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,first_ord,f):
  from sklearn.model_selection import train_test_split

  from sklearn.linear_model import BayesianRidge
  from joblib import dump, load
  
  if tun==1:
    
    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score
    
    br_model = BayesianRidge()

    params = {
        'n_iter': (100, 300),
        'tol': (1e-6, 1e-3, 'log-uniform'),
        'alpha_1': (1e-9, 1e-5, 'log-uniform'),
        'alpha_2': (1e-9, 1e-5, 'log-uniform'),
        'lambda_1': (1e-9, 1e-5, 'log-uniform'),
        'lambda_2': (1e-9, 1e-5, 'log-uniform'),
    }

    br = BayesSearchCV(br_model, params, n_iter=tun_iter, cv=5)

    br.fit(X_train, y_train)

    score = cross_val_score(br, X_train, y_train, cv=5).mean()
    
    if os.path.exists(f'{output_path}tuning_results_br_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_br_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_br_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(br.best_params_))
        file.write("Cross-validation score: {}\n".format(score))
    
    dump(br, output_path+'br_model_'+f[:-4]+'.joblib')
    y_pred = br.predict(X_test)
    
  else:    
    if tun==2:
      br = load(output_path+'br_model_'+f[:-4]+'.joblib')
      y_pred = br.predict(X_test)
    else:
      br = BayesianRidge()
      br.fit(X_train, y_train)
      
      y_pred = br.predict(X_test)

  ''
  from SALib.sample import sobol
  X_sobol = sobol.sample(problem, N, calc_second_order=True)
  from numpy import zeros
  Y_sobol = zeros((len(X_sobol), 1))
  for i in range(len(X_sobol)):
      Y_sobol[i] = br.predict(X_sobol[i].reshape(1, -1))
  Y_sobol = Y_sobol.reshape(-1)
  from SALib.analyze import sobol
  Si = sobol.analyze(problem, Y_sobol, calc_second_order=True, print_to_console=False)
  ''

  importances = Si['ST']
  first_order=Si['S1']

  interactions=Si['S2']
  np.savetxt(output_path+f'interactions_matrix_{len(importance_list)}.txt', interactions, delimiter='\t', fmt='%f')
  
  if len(importance_list) <= totalhours-1:
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances       
      importance_list.append(normalized_importances)
      #importance_list.append(importances)
      first_ord.append(first_order)
  else:
      importance_list = []
      first_ord=[]


  from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value  
  
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[] 

def sa_gaussianreg(X_train, X_test, y_train, y_test,problem,N, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,first_ord,f):

  from sklearn.gaussian_process import GaussianProcessRegressor
  from sklearn.gaussian_process.kernels import RBF
  from joblib import dump, load
  
  if tun==1:
    
    from skopt import BayesSearchCV
    from sklearn.model_selection import cross_val_score  
    
    kernel = RBF(length_scale=1.0)
    gp_model = GaussianProcessRegressor(kernel=kernel)

    params = {
        'alpha': (1e-10, 1e+1, 'log-uniform'),
        'n_restarts_optimizer': (0, 5),
    }

    gp = BayesSearchCV(gp_model, params, n_iter=tun_iter, cv=5)

    gp.fit(X_train, y_train)

    score = cross_val_score(gp, X_train, y_train, cv=5).mean()

    if os.path.exists(f'{output_path}tuning_results_gp_{f[:-4]}.txt'):
        os.remove(f'{output_path}tuning_results_gp_{f[:-4]}.txt')
    with open(f'{output_path}tuning_results_gp_{f[:-4]}.txt', 'a') as file:
        file.write("Best parameters: {}\n".format(gp.best_params_))
        file.write("Cross-validation score: {}\n".format(score))
    
    dump(gp, output_path+'gp_model_'+f[:-4]+'.joblib')
    
    y_pred = gp.predict(X_test)
        
  else:  
    
    if tun==2:
      gp = load(output_path+'gp_model_'+f[:-4]+'.joblib')
      y_pred = gp.predict(X_test)
    else:
      kernel = RBF(length_scale=1.0)
      gp = GaussianProcessRegressor(kernel=kernel)
      gp.fit(X_train, y_train)
      
      
      y_pred = gp.predict(X_test)

  ''
  from SALib.sample import sobol
  X_sobol = sobol.sample(problem, N, calc_second_order=True)
  from numpy import zeros
  Y_sobol = zeros((len(X_sobol), 1))
  for i in range(len(X_sobol)):
      Y_sobol[i] = gp.predict(X_sobol[i].reshape(1, -1))
  Y_sobol = Y_sobol.reshape(-1)
  from SALib.analyze import sobol
  Si = sobol.analyze(problem, Y_sobol, calc_second_order=True, print_to_console=False)
  ''
  
  importances = Si['ST']
  first_order=Si['S1']

  interactions=Si['S2']
  np.savetxt(output_path+f'interactions_matrix_{len(importance_list)}.txt', interactions, delimiter='\t', fmt='%f')
    
  
  if len(importance_list) <= totalhours-1:
      sum_importances = np.sum(importances, axis=0)
      normalized_importances = importances / sum_importances       
      importance_list.append(normalized_importances)
      #importance_list.append(importances)
      first_ord.append(first_order)
  else:
      importance_list = []
      first_ord=[]

  from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
  
  spearman_corr, p_value = spearmanr(y_test, y_pred)
  score=spearman_corr
  pvalue= p_value  
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  
  if len(score_list) <= totalhours-1:
    score_list.append(score)
    pvalue_list.append(pvalue)
    mse_list.append(mse)
    mae_list.append(mae)
  else:
    score_list=[]
    pvalue_list=[]
    mse_list=[]
    mae_list=[]

  if len(ytest)<=totalhours-1:
    ytest.append(y_test)
    ypred.append(y_pred)
  else:
    ytest=[]
    ypred=[]



#############################################################################################
import ipywidgets as widgets
from IPython.display import display

def create_predict_and_plot_global(meth,N,var,hpoint,vpoint,hour,tun, n_sample, figsize=(10,6)):
  def predict_and_plot(**x_dict):

    if var >= 1 and var <= len(variables):
        nam1 = variables[var - 1]
    else:
        nam1 = 'Invalid var value'

    if hpoint >= 1 and hpoint <= len(regions):
        nam2 = regions[hpoint - 1]
    else:
        nam2 = 'Invalid hpoint value'        

    name=nam1+'_'+nam2+'_lev'+str(vpoint)
    file_list=[nam1+'_'+nam2+'_lev'+str(vpoint)+'_'+str(i)+'.txt' for i in range(1,totalhours+1)]

    Xnonscaled = np.loadtxt(output_path+'X.txt') 


    for file in file_list:

      f=file
      ynonscaled = np.loadtxt(output_path+file, delimiter=',')
      
      
      if N==int(totalsim*1/10):
        Xnonscaled = Xnonscaled[:int(totalsim*1/10),:]
        ynonscaled = ynonscaled[:int(totalsim*1/10)]
      elif N==int(totalsim*2/10):
        Xnonscaled = Xnonscaled[:int(totalsim*2/10),:]
        ynonscaled = ynonscaled[:int(totalsim*2/10)]
      elif N==int(totalsim*3/10):
        Xnonscaled = Xnonscaled[:int(totalsim*3/10),:]
        ynonscaled = ynonscaled[:int(totalsim*3/10)]
      elif N==int(totalsim*4/10):
        Xnonscaled = Xnonscaled[:int(totalsim*4/10),:]
        ynonscaled = ynonscaled[:int(totalsim*4/10)] 
      elif N==int(totalsim*5/10):
        Xnonscaled = Xnonscaled[:int(totalsim*5/10),:]
        ynonscaled = ynonscaled[:int(totalsim*5/10)]
      elif N==int(totalsim*6/10):
        Xnonscaled= Xnonscaled[:int(totalsim*6/10),:]
        ynonscaled = ynonscaled[:int(totalsim*6/10)]
      elif N==int(totalsim*7/10):
        Xnonscaled= Xnonscaled[:int(totalsim*7/10),:]
        ynonscaled = ynonscaled[:int(totalsim*7/10)]
      elif N==int(totalsim*8/10):
        Xnonscaled= Xnonscaled[:int(totalsim*8/10),:]
        ynonscaled = ynonscaled[:int(totalsim*8/10)]
      elif N==int(totalsim*9/10):
        Xnonscaled= Xnonscaled[:int(totalsim*9/10),:]
        ynonscaled = ynonscaled[:int(totalsim*9/10)]
      elif N==int(totalsim*10/10):
        Xnonscaled= Xnonscaled[:int(totalsim*10/10),:]
        ynonscaled = ynonscaled[:int(totalsim*10/10)]
      
           
      y=ynonscaled
      X=Xnonscaled              

      from sklearn.model_selection import train_test_split
      
      partition=0.3
        
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=partition, random_state=42)

      from sklearn.preprocessing import StandardScaler
      from joblib import dump

      scalerX = StandardScaler()
      X_train = scalerX.fit_transform(X_train)
      X_test = scalerX.transform(X_test)

      scalery = StandardScaler()
      y_train = scalery.fit_transform(y_train.reshape(-1, 1)).ravel()
      y_test = scalery.transform(y_test.reshape(-1, 1)).ravel()
      
      ''
      Xlow = np.min(X_train, axis=0)
      Xup = np.max(X_train, axis=0)
      Nn = n_sample 
      bounds = [(Xlow[i], Xup[i]) for i in range(Xlow.shape[0])]
      problem = {'num_vars': X.shape[1], 'names': parameter_names, 'bounds': bounds}            
      ''

      if meth==1:
        Si = sa_randomforest(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f)
        method='randomforest'
      elif meth==2:
        Si = sa_lassoregression(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f)
        method='lasso'
      elif meth==3:
        Si = sa_svm(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f)
        method='svm'
      elif meth==4:        
        Si = sa_baesyanreg(X_train, X_test, y_train, y_test,problem,Nn, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,first_ord,f)
        method='br'
      elif meth==5:
        Si = sa_gaussianreg(X_train, X_test, y_train, y_test,problem,Nn, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,first_ord,f)
        method='gp'
      elif meth==6:
        Si = sa_xgboost(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f)
        method='xgb'
      elif meth==7:
        Si = sa_cart(X_train, X_test, y_train, y_test, importance_list,score_list,pvalue_list,mse_list,mae_list,ytest,ypred,tun,f)
        method='cart'                
        
      data = np.array([score_list, pvalue_list, mse_list, mae_list]).T
      
      import pandas as pd
      df = pd.DataFrame(data, columns=['score', 'pvalue' , 'mse', 'mae'])
      importance_df = pd.DataFrame(importance_list)
      if hour==totalhours:
        importance_df.to_csv(output_path+'importance'+method+str(N)+file[:-7]+'.txt', header=False, index=False, sep=' ')
        df.to_csv(output_path+'df'+method+str(N)+file[:-7]+'.txt', header=False, index=False, sep=' ')

#############################################################################################
    import matplotlib
    import matplotlib.pyplot as plt

    matplotlib.rc('xtick',labelsize=10)
    matplotlib.rc('ytick',labelsize=10)
    
    x=np.arange(totalhours)

    pvalue = df.loc[hour-1, 'pvalue']
    if pvalue < 0.0001:
        pvalue_str = "<0.0001"
    else:
        pvalue_str = "{:.3g}".format(pvalue)

    
    font=12
    
    if meth in [4,5]:
    
      fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12, 12))

      x = np.arange(totalhours)
      
      from matplotlib.cm import get_cmap
      cmap = get_cmap('viridis')
      colors = [cmap(i / (len(parameter_names)-1)) for i in range(len(parameter_names))]
      #colors = ['b','g','r','c','m','y']
      colors = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#999999']
      for i in range(len(parameter_names)):     
        axs[0,0].plot(x,[abs(importance_list[j][i]) for j in range(totalhours)],label=parameter_names[i],color=colors[i])  
      
      axs[0,0].set_xlabel("Time",fontsize=12)
      axs[0,0].set_ylabel("Importance",fontsize=12)
      axs[0,0].set_title('Importance time evolution',fontsize=12)
      axs[0,0].legend(prop={'size':10, 'weight':'normal'}, ncol=4, loc='upper center', bbox_to_anchor=(0.5, 0.99))
      axs[0,0].grid(True)

      ln1 =axs[0,1].plot(df.index, df['score'], color=colors[0], label='Score')
      ln2 =axs[0,1].plot(df.index, df['mse'], color=colors[1], label='MSE')
      ln3 =axs[0,1].plot(df.index, df['mae'], color=colors[2], label='MAE')
      twin=axs[0,1].twinx()
      ln4 =twin.plot(df.index, df['pvalue'], color=colors[3], label='p-value')
      twin.set_ylabel('p-value',fontsize=12)
      twin.set_ylim(0, 1e-15)
      axs[0,1].set_xlabel('Time',fontsize=12)
      axs[0,1].set_ylabel('Value',fontsize=12)
      axs[0,1].set_title('Metrics time evolution',fontsize=12)
      axs[0,1].set_ylim(0, 1)
      lns = ln1 + ln2 + ln3 + ln4
      labels = [l.get_label() for l in lns]
      axs[0,1].legend(lns,labels,prop={'size':10,'weight':'normal'},loc='upper center', bbox_to_anchor=(0.15, 0.8))
      axs[0,1].grid(True)

      axs[1,0].scatter(ytest[hour-1], ypred[hour-1])
      ideal = [min(ytest[hour-1]), max(ytest[hour-1])]
      axs[1,0].plot(ideal, ideal, 'r--')
      axs[1,0].set_xlabel('True Values',fontsize=12)
      axs[1,0].set_ylabel('Predictions',fontsize=12)
      axs[1,0].set_title('Hour: {}; Performance (score: {:.2f} [p: {}], mse: {:.2f}, mae: {:.2f})'.format(hour-1, df.loc[hour-1, 'score'], pvalue_str, df.loc[hour-1, 'mse'], df.loc[hour-1, 'mae']))
      axs[1,0].grid(True)

      axs[1,1].bar(parameter_names,importance_list[hour-1])
      axs[1,1].set_xticklabels(parameter_names, rotation=90, ha='right')
      axs[1,1].set_xlabel("Features",fontsize=12)
      axs[1,1].set_ylabel("Coefficient",fontsize=12)
      axs[1,1].set_title('Hour: '+str(hour-1)+'; Sobol total index',fontsize=12)
      axs[1,1].grid(True)

      import seaborn as sns
      file_path = os.path.join(output_path, f'interactions_matrix_{hour-1}.txt')
      inter_mat = np.loadtxt(file_path, delimiter='\t')
      sns.heatmap(inter_mat, annot=True, cmap='coolwarm', fmt=".3f", cbar_kws={'label': 'Sobol second order index'},
                  xticklabels=parameter_names, yticklabels=parameter_names, ax=axs[2, 0])
      axs[2, 0].set_title(f'Interactions Matrix {hour-1}',fontsize=12)
      
      
      axs[2,1].bar(parameter_names,first_ord[hour-1])
      axs[2,1].set_xticklabels(parameter_names, rotation=90, ha='right')
      axs[2,1].set_xlabel("Features",fontsize=12)
      axs[2,1].set_ylabel("Coefficient",fontsize=12)
      axs[2,1].set_title('Hour: '+str(hour-1)+'; Sobol first order index',fontsize=12)    
      axs[2,1].grid(True)

    else:
      fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

      x = np.arange(totalhours)
      
      from matplotlib.cm import get_cmap
      cmap = get_cmap('viridis')
      
      
      colorsfill = plt.cm.tab10(np.linspace(0, 1, 6))
      colors = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#999999']
      for i in range(len(parameter_names)):    
        axs[0,0].plot(x,[abs(importance_list[j][i]) for j in range(totalhours)],label=parameter_names[i],color=colors[i]) 
      
      axs[0,0].set_xlabel("Time",fontsize=12)
      axs[0,0].set_ylabel("Importance",fontsize=12)
      axs[0,0].set_title('Importance time evolution',fontsize=12)
      axs[0,0].set_ylim(0, 1)
      axs[0,0].legend(prop={'size':10, 'weight':'normal'}, ncol=4, loc='upper center', bbox_to_anchor=(0.5, 0.99))
      axs[0,0].grid(True)
      
      ln1 =axs[0,1].plot(df.index, df['score'], color=colors[0], label='Score')
      ln2 =axs[0,1].plot(df.index, df['mse'], color=colors[1], label='MSE')
      ln3 =axs[0,1].plot(df.index, df['mae'], color=colors[2], label='MAE')
      twin=axs[0,1].twinx()
      ln4 =twin.plot(df.index, df['pvalue'], color=colors[3], label='p-value')
      twin.set_ylabel('p-value',fontsize=12)
      twin.set_ylim(0, 1e-15)
      axs[0,1].set_xlabel('Time',fontsize=12)
      axs[0,1].set_ylabel('Value',fontsize=12)
      axs[0,1].set_title('Metrics time evolution',fontsize=12)
      axs[0,1].set_ylim(0, 1)
      lns = ln1 + ln2 + ln3 + ln4
      labels = [l.get_label() for l in lns]
      axs[0,1].legend(lns,labels,prop={'size':10,'weight':'normal'},loc='upper center', bbox_to_anchor=(0.15, 0.8))
      axs[0,1].grid(True)
      
      axs[1,0].scatter(ytest[hour-1], ypred[hour-1])
      ideal = [min(ytest[hour-1]), max(ytest[hour-1])]
      axs[1,0].plot(ideal, ideal, 'r--')
      axs[1,0].set_xlabel('True Values',fontsize=12)
      axs[1,0].set_ylabel('Predictions',fontsize=12)
      axs[1,0].set_title('Hour: {}; Performance (score: {:.2f} [p: {}], mse: {:.2f}, mae: {:.2f})'.format(hour-1, df.loc[hour-1, 'score'], pvalue_str, df.loc[hour-1, 'mse'], df.loc[hour-1, 'mae']))
      axs[1,0].grid(True)
      
      axs[1,1].bar(parameter_names,abs(importance_list[hour-1]))
      
      axs[1,1].set_xlabel("Features",fontsize=12)
      axs[1,1].set_xticklabels(parameter_names, rotation=90, ha='right')
      axs[1,1].set_ylabel("Coefficient",fontsize=12)
      axs[1,1].set_title('Hour: '+str(hour-1)+'; Feature importance',fontsize=12)
      axs[1,1].grid(True)
      
    fig.suptitle(method + ' on ' + name[:-9] + ' N:'+ str(N) + ' v:' + str(vpoint), fontsize=16)
    plt.tight_layout()
    plt.savefig('C:/Users/dario/Downloads/' +method+'_'+name[:-9] +'_hour'+str(hour-1)+ '_v'+str(vpoint)+'.png', dpi=300)
    plt.show()



  return predict_and_plot
########################################################################

def nearest_divisible(n , lst):
  size = len(lst)
  remainder = n % (size+2)
  if remainder == 0:
    return n
  diff = size - remainder
  return n + diff

n_sample_calc = nearest_divisible(1000, parameter_names)

choices=['vpoint','hpoint','variable','N_simul','method','hour','tuning']
vpointmin=1;
vpointmax=verticalmax;
hpointmin=1;
hpointmax=len(regions);
varmin=1;
varmax=len(variables);
Nmin=totalsim*1/10;
Nmax=totalsim;
methmin=1;
methmax=7; ###
tunmin=0;
tunmax=2;
hourmin=1;
hourmax=totalhours;
Xmin=[vpointmin,hpointmin,varmin,Nmin,methmin,hourmin,tunmin]
Xmax=[vpointmax,hpointmax,varmax,Nmax,methmax,hourmax,tunmax]
Nstep=[verticalmax-1,hpointmax-1,varmax-1, (Nmax - Nmin) // 10, methmax-1, totalhours-1 , tunmax]

def interactive_global(on_change_global,meth,N,var,hpoint,vpoint,hour,tun):

    Xnew=[vpoint,hpoint,var,N,meth,hour,tun]

    sliders = {}
    for i, f in enumerate(choices):
        sliders[f] = widgets.FloatSlider(
            value=Xnew[i],
            min=Xmin[i],
            max=Xmax[i],
            step=(Xmax[i] - Xmin[i]) / Nstep[i],
            description=f"{f}",
            readout_format=".1f",
            layout={'width': '500px', 'margin': '0px 0px 0px 20px'}
        )


    # setup ui
    out = widgets.interactive_output(on_change_global, sliders)    
    controls = widgets.VBox(list(sliders.values()))
    controls.layout = widgets.Layout(width='50%', align_items='center', justify_content='space-between')
    title = widgets.HTML("<h1 style='margin-top: 20px; text-align: center; font-family: Arial, sans-serif; font-size: 36px;'>ML-AMPSIT</h1>")
    subtitle = widgets.HTML("<h2 style='margin-bottom: 20px; text-align: center;'>Machine Learning-based Automated Multi-method Parameter Sensitivity and Importance analysis Tool</h2>")
    subtitle2 = widgets.HTML("<h3 style='margin-bottom: 5px; text-align: center;'>interactive UI made by Dario Di Santo, Department of Civil, Environmental and Mechanical Engineering, University of Trento, Trento, Italy</h3>")
    subtitle3 = widgets.HTML("<h4 style='margin-bottom: 5px; text-align: center;'>How the script works: The first time it is used, option 1 (initialization) must be set to correctly initialize all variables; then change the value of the sliders on the UI and run the script again with option 2 (customization). This will re-run the methods with the custom values. All subsequent customizations must be run with option 2. More about this work at the following link: <a href='https://github.com/ML-AMPSIT/ML-AMPSIT'>Github repository</a> </h4>")
    label3_text = "<b>variable:</b> " + ", ".join(variables)
    label3 = widgets.HTML(label3_text)
    label4 = widgets.HTML('<b>Number of simulations:</b> change simulations amount for convergence analysis')
    label5 = widgets.HTML('<b>method:</b> (1)Random Forest, (2)Lasso, (3)Support Vector Regression, (4)Bayesian regression, (5)Gaussian regression, (6)XGBoost, (7)CART')
    label1_text = f"<b>vertical levels:</b> {verticalmax}"
    label1 = widgets.HTML(label1_text)
    label2_text = "<b>domain horizontal point:</b> " + ", ".join(regions)
    label2 = widgets.HTML(label2_text)
    label6 = widgets.HTML('<b>hour:</b> select an hour of interest to show associated metrics and importance ranking')
    label7 = widgets.HTML('<b>tun:</b> tuning option: (0)off, (1)on, (2)load')
    ui = widgets.VBox([widgets.HTML('<br>'), title, subtitle,subtitle2,subtitle3, widgets.HTML('<br>'), 
                      widgets.HBox([controls,widgets.VBox([label1, label2, label3, label4, label5, label6, label7])]),
                      widgets.HTML('<br>'), out])
    display(ui)
    return sliders


while True:
    init = input("initialization=1 , customization=2 : ")
    if init == "1" or init == "2":
        break
    print("Valore non valido, inserisci 1 o 2.")
if init == "1":  
  on_change_global=create_predict_and_plot_global(meth=1, N=totalsim, var=1, hpoint=1, vpoint=1, hour=totalhours, tun=0, n_sample=n_sample_calc)
  sliders=interactive_global(on_change_global,meth=1, N=totalsim, var=1, hpoint=1, vpoint=1, hour=totalhours, tun=0)  

elif init=="2":
  meth = int(sliders['method'].value)
  N = int(sliders['N_simul'].value)
  var = int(sliders['variable'].value)
  hpoint = int(sliders['hpoint'].value)
  vpoint = int(sliders['vpoint'].value)
  hour = int(sliders['hour'].value)
  tun = int(sliders['tuning'].value)

  on_change_global=create_predict_and_plot_global(meth=meth, N=N, var=var, hpoint=hpoint,vpoint=vpoint,hour=hour, tun=tun, n_sample=n_sample_calc)

  sliders=interactive_global(on_change_global,meth=meth, N=N, var=var, hpoint=hpoint, vpoint=vpoint, hour=hour, tun=tun)

